# Text Generation Using n-grams

In this example, we will extract and count n-grams features from documents, and then use them to generate text. 

The example is based on the [Python for NLP: Developing an Automatic Text Filler using n-grams example by Usman Malik](https://stackabuse.com/python-for-nlp-developing-an-automatic-text-filler-using-n-grams/).

In [1]:
import re
import nltk
import numpy as np
import random
import string
import bs4 as bs
import urllib.request

## Prepare the Data

Below, we will read a set of Wikipedia articles related to robotics and AI (feel free to change it). 

After some regular expression based preprocessing to convert the data to lowercase alphanumeric with basic punctuation, we can extract features from this text.

In [2]:
# Read a Wikipedia page and combine all the paragraphs' text
articles = ["Robotics", "Artificial_intelligence", "Machine_learning", "Computer_vision", 
            "Human-robot_interaction", "Robotic_sensing", "Robotic_sensors", "Cyber-physical_system", 
            "Robot_locomotion", "Mobile_robot", "Robotic_mapping", "Robotic_manipulation"]
article_text = ""
for article_name in articles:
    raw_html = urllib.request.urlopen("https://en.wikipedia.org/wiki/" + article_name)
    raw_html = raw_html.read()
    article_html = bs.BeautifulSoup(raw_html, "lxml")
    article_paragraphs = article_html.find_all("p")
    for para in article_paragraphs:
        article_text += para.text

# Convert all text to lower case and remove anything besides alphanumerics, some punctuation, and space
article_text = article_text.lower()
article_text = re.sub(r"[-]", " ", article_text)
article_text = re.sub(r"(?P<n1>[A-Za-z])[^A-za-z ](?P<n2>[A-za-z])", "\g<n1> \g<n2>", article_text)
article_text = re.sub(r"[^A-Za-z., ]", "", article_text)

# Print the first few words
num_words = len(nltk.word_tokenize(article_text))
print("Total number of words: {}\n".format(num_words))
print(article_text[:1000])

Total number of words: 50476

robotics is an interdisciplinary research area at the interface of computer science and engineering. robotics involves design, construction, operation, and use of robots. the goal of robotics is to design intelligent machines that can help and assist humans in their day to day lives and keep everyone safe. robotics draws on the achievement of information engineering, computer engineering, mechanical engineering, electronic engineering and others.robotics develops machines that can substitute for humans and replicate human actions. robots can be used in many situations and for many purposes, but today many are used in dangerous environments including inspection of radioactive materials, bomb detection and deactivation, manufacturing processes, or where humans cannot survive e g. in space, underwater, in high heat, and clean up and containment of hazardous materials and radiation. robots can take on any form but some are made to resemble humans in appearance

## Extract n-gram Features

Below we will extract n-grams from the documents from scratch.

Each unique n-gram (e.g. the trigram "robotics is an") is stored as a Python dictionary key, where its values are a list of all the words following that sequence as seen in the entire dataset. In theory, reading an infinitely large dataset will give us perfect probability distribution for generating the next word given any n-gram.

However, note that NLTK has several utilities for extracting n-grams, padding sequences, and more. You can see good example from [this notebook on Kaggle by Liling Tan](https://www.kaggle.com/alvations/n-gram-language-model-with-nltk).

In [3]:
def get_training_ngrams(text, N):
    """ Extracts n-grams as well as the next word in the sequence for later sampling"""
    ngrams = {}
    words_tokens = nltk.word_tokenize(text)
    for i in range(len(words_tokens)-N):
        seq = " ".join(words_tokens[i:i+N])
        if seq not in ngrams.keys():
            ngrams[seq] = []
        ngrams[seq].append(words_tokens[i+N])
    return ngrams


# Gather n-grams plus the next possible words for completion
N = 3
ngrams = get_training_ngrams(article_text, N)

# Print some random n-grams that have at least 1, 2, and 3 completions respectively
num_samples = 5
generated_keys = 5
print("Sample {}-grams:".format(N))
for num_completions in range(3):
    generated_keys = 0
    while generated_keys < num_samples:
        key = random.choice(list(ngrams.keys()))
        if len(ngrams[key]) > num_completions:
            print("{}: {}".format(key, ngrams[key]))
            generated_keys += 1

Sample 3-grams:
natural , human: ['esque']
university , stanford: ['and']
motion estimation ,: ['visual']
can make choices: ['and']
wing of the: ['robot']
about a certain: ['measurement', 'measurement']
designed recently ,: ['such', 'such']
, thus avoiding: ['workers', 'workers']
are many competitions: ['around', 'around']
understanding digital images: [',', ',']
takes actions that: ['maximize', 'maximize', 'maximize']
in the next: ['couple', 'few', 'two']
, the robots: ['must', 'first', 'must']
information can be: ['used', 'used', 'used']
through unsupervised learning: [',', 'in', ',']


## Generate Text using the n-gram counts

For this relatively small dataset (n-grams are VERY data hungry), the assumption that our dictionary will hold a reasonable probability distribution of next words to sample from will be far from true. 

With such limited data, our approach will be simple -- given an n-gram, we will look it up in our dictionary and sample from its list of subsequent words at random.

In [4]:
def generate_sequence(start, ngrams, num_words):
    """ 
    Generate a sequence given start text, the dictionary of n-grams, 
    and total number of words before stopping. If no valid word is found, 
    the sequence generation will terminate early.
    """
    curr_sequence = start.lower()
    output = curr_sequence
    N = len(list(ngrams.keys())[0].split(" "))
    for i in range(num_words):
        if curr_sequence not in ngrams.keys():
            print("No valid word following the sequence: " + curr_sequence)
            break
        possible_words = ngrams[curr_sequence]
        next_word = random.choice(possible_words)
        output += " " + next_word
        seq_words = nltk.word_tokenize(output)
        curr_sequence = " ".join(seq_words[len(seq_words)-N:len(seq_words)])
    return output


start_text = "Robotics is a"
num_words = 100
gen_output = generate_sequence(start_text, ngrams, num_words)

print("{}-grams:".format(N))
print("="*30)
print(gen_output)

3-grams:
robotics is a branch of engineering that involves the conception , design , manufacture , and operation of robots . for example , the authors have examined the task of digital character recognition , the mnist dataset has often been used . machine learning algorithms that can be used for various problems reasoning using the bayesian inference algorithm , learning using the expectation maximization algorithm , f planning using decision networks and perception using dynamic bayesian networks . generalizations of bayesian networks that can represent and solve decision problems under uncertainty are called influence diagrams.a genetic algorithm ga is a search that is


## Try With Other Sizes

The value of "n" in n-grams provides a tradeoff: Longer sequences will lead to more natural text since we have more context. However, the longer the sequence, the less likely it is to encounter it again when generating text, so you will need more data and more luck. This is why n-grams become impractical after 4- or 5-grams, and most practical models use bigrams or trigrams.

In [5]:
N = 2
num_words = 100
start_text = "Computer vision"
ngrams = get_training_ngrams(article_text, N)
gen_output = generate_sequence(start_text, ngrams, num_words)
print("{}-grams:".format(N))
print("="*30)
print(gen_output)

print("")

N = 4
num_words = 100
start_text = "Machine learning models require"
ngrams = get_training_ngrams(article_text, N)
gen_output = generate_sequence(start_text, ngrams, num_words)
print("{}-grams:".format(N))
print("="*30)
print(gen_output)

# Funny thing about the paragraph below regarding overfitting
# https://en.wikipedia.org/wiki/Machine_learning#Training_models

2-grams:
computer vision share other topics such as jumping , and building new robots serve various practical purposes , but this will not happen until the th century author isaac asimov introduced the three laws of robotics , fast implementations of cnns on gpus have won many visual pattern recognition and information value theory these tools include models such as naive bayes on most practical data sets for tasks ranging from advanced missiles to uavs for recon missions or missile guidance send the missile to an increase in research tools and people , and the s , p ot at oe sb ur

4-grams:
machine learning models require a lot of data in order for them to perform well . usually , when training a machine learning model.federated learning is an adapted form of distributed artificial intelligence to training machine learning models that decentralizes the training process , allowing for users privacy to be maintained by not needing to send their data to a centralized server . this also i

## Combine Multiple Values of "n"

In practice, we can combine the advantages of longer sequences with shorter sequences by collecting multiple sets of n-grams (for example, bigrams and trigrams).

At inference time, we can try generate with the longer sequences first, and as a fallback move down to the shorter sequences since generating *some* text is better than exiting early, even if accuracy is not as good... or use a weighted combination where longer sequences are weighted more heavily for sampling. This also lets us sample much longer sequences!

Of course, this exponentially increases the memory requirements of our dictionary... no free lunch!

In [6]:
def generate_sequence_multiple(start, all_grams, all_probs, num_words):
    """ 
    Generate a sequence given start text, a list of n-grams dictionaries, 
    and total number of words before stopping. If no valid word is found, 
    the sequence generation will terminate early.
    """
    curr_sequence = start.lower()
    output = curr_sequence
    N_list = [len(list(ngrams.keys())[0].split(" ")) for ngrams in all_grams]
    
    for i in range(num_words):
        possible_words = []
        sample_probs = []
        seq_words = nltk.word_tokenize(output)
        
        # Loop through all the n-grams dictionaries
        for idx in range(len(N_list)):
            ngrams = all_grams[idx]
            N = N_list[idx]
            
            # If the sequence is not found, fall back and try again until 
            # all n-grams dictionaries have been exhausted
            if len(seq_words) >= N:    
                curr_sequence = " ".join(seq_words[len(seq_words)-N:len(seq_words)])
                if curr_sequence in ngrams.keys():
                    words = ngrams[curr_sequence]
                    possible_words.extend(words)
                    sample_probs.extend(all_probs[idx]*np.ones(len(words)))
                    
        # If there are no possible words, exit
        if len(possible_words) == 0:
            break
        
        # Generate text
        sample_probs /= sum(sample_probs)
        next_word = np.random.choice(possible_words, p=sample_probs)
        output += " " + next_word
            
    return output

bigrams = get_training_ngrams(article_text, 2)
trigrams = get_training_ngrams(article_text, 3)
fourgrams = get_training_ngrams(article_text, 4)
all_grams = [bigrams, trigrams, fourgrams]
all_probs = [0.05, 0.15, 0.8]

start_text = "Robotics is"
num_words = 500
gen_output = generate_sequence_multiple(start_text, all_grams, all_probs, num_words)

print("multi-gram combination:".format(N))
print("="*30)
print(gen_output)

multi-gram combination:
robotics is a related field that considers any kind of programming languages for representing hypotheses and not only logic programming , such as the imagenet challenge , promote research in ai research began to explore the possibility that human intelligence can be smaller and lighter without a human pilot on board , and faithfully reproduce the author s original intent social intelligence . a problem like machine translation is considered ai complete , because all of these problems need to combine optimal skills has resulted in collaborative robots and humans sharing a common workspace more closely and led to the development of new customer groups that are not represented in the spring of , the robot s onboard computer tries to keep the total inertial forces the combination of earth s gravity and the robotic holograms in voyager.are there limits to how intelligent machines or human machine hybrids can be thought of as a predictive model to go from observations